In [1]:
import urllib.request as req
import zipfile
import os.path 
import MeCab
from gensim import models

In [2]:
mecab = MeCab.Tagger()
mecab.parse("")

'EOS\n'

In [3]:
model = models.Doc2Vec.load('./model/aozora.model')

In [4]:
def read_book(url, zipname):
    if not os.path.exists(zipname):
        req.urlretrieve(url, zipname)

    with zipfile.ZipFile(zipname,"r") as zf:
        for filename in zf.namelist():
            with zf.open(filename,"r") as f:
                return f.read().decode("shift-jis")

In [5]:
def split_words(text):
    node = mecab.parseToNode(text)
    wakati_words = []
    while node is not None:
        hinshi = node.feature.split(",")[0]
        if  hinshi in ["名詞"]:
            wakati_words.append(node.surface)
        elif hinshi in ["動詞", "形容詞"]:
            wakati_words.append(node.feature.split(",")[6])
        node = node.next
    return wakati_words

In [6]:
def similar(title, url):
    zipname = url.split("/")[-1]
        
    words = read_book(url, zipname)
    wakati_words = split_words(words)
    vector = model.infer_vector(wakati_words)
    print("--- 「" + title + '」 と似た作品は? ---')
    print(model.docvecs.most_similar([vector],topn=3))
    print("")

In [7]:
similar("宮沢 賢治:よだかの星",
        "https://www.aozora.gr.jp/cards/000081/files/473_ruby_467.zip")

similar("芥川 龍之介:犬と笛",
        "https://www.aozora.gr.jp/cards/000879/files/56_ruby_845.zip")

similar("ポー エドガー・アラン:マリー・ロジェエの怪事件",
        "https://www.aozora.gr.jp/cards/000094/files/4261_ruby_54182.zip")

similar("紫式部:源氏物語 06 末摘花",
        "https://www.aozora.gr.jp/cards/000052/files/5021_ruby_11106.zip")

--- 「宮沢 賢治:よだかの星」 と似た作品は? ---
[('宮澤 賢治:やまなし', 0.9145612716674805), ('宮澤 賢治:セロ弾きのゴーシュ', 0.8463627696037292), ('宮澤 賢治:どんぐりと山猫', 0.826591432094574)]

--- 「芥川 龍之介:犬と笛」 と似た作品は? ---
[('芥川 竜之介:老年', 0.7856045961380005), ('紫式部:源氏物語 05 若紫', 0.7021193504333496), ('芥川 竜之介:鼻', 0.6975603699684143)]



<ipython-input-6-5af90f8f78f2>:8: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  print(model.docvecs.most_similar([vector],topn=3))


--- 「ポー エドガー・アラン:マリー・ロジェエの怪事件」 と似た作品は? ---
[('ポー エドガー・アラン:ウィリアム・ウィルスン', 0.9013032913208008), ('ポー エドガー・アラン:黒猫', 0.8486806154251099), ('ポー エドガー・アラン:落穴と振子', 0.7110996246337891)]

--- 「紫式部:源氏物語 06 末摘花」 と似た作品は? ---
[('紫式部:源氏物語 05 若紫', 0.9307647347450256), ('紫式部:源氏物語 03 空蝉', 0.907958984375), ('紫式部:源氏物語 02 帚木', 0.8453316688537598)]

